In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from scipy import ndimage
from skimage.morphology import label
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Input data files are available in the "../input/" directory.
#Simple CNN keras
from keras.layers import Lambda
from keras.models import Model , load_model
from keras.models import Sequential
from keras.layers import concatenate
from keras.layers import Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D 
from keras.losses import binary_crossentropy
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Conv2DTranspose
from keras import backend as K

import tensorflow as tf
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
from subprocess import check_output
import os
print(check_output(["ls", "../input"]).decode("utf8"))
from skimage.transform import rescale, resize, downscale_local_mean
import tensorflow as tf
# Any results you write to the current directory are saved as output.

In [ ]:
train_dir = "../input/stage1_train/"

In [ ]:
train_images = [i for i in os.listdir(train_dir)]

In [ ]:
final_images = []

for dirs in train_images:
    final_images.append(plt.imread(train_dir + dirs + '/' + 'images/' + dirs+ '.png'))

    

In [ ]:
label_masks = []
for dirs in train_images:
    for i,img in enumerate(os.listdir(train_dir + dirs + "/" + "masks/")):
        if i == 0:
            maximum = plt.imread(train_dir + dirs + "/" + "masks/" + img)
            #time_pass = plt.imread(train_dir + dirs + "/" + "masks/" + img)
        else:
            image = plt.imread(train_dir + dirs + "/" + "masks/" + img)
            maximum = np.maximum(maximum,image)
            #print(maximum.shape)
    label_masks.append(maximum)

In [ ]:
label_masks = np.array(label_masks)

In [ ]:
final_images = np.array(final_images)

In [ ]:
plt.figure(1.,figsize = (10,10))
#plt.subplot(2,1,1)
#plt.imshow(label_masks[20])
plt.subplot(2,1,1) 
plt.imshow(final_images[20])

In [ ]:
label_masks[20].shape

In [ ]:
plt.figure(1)
plt.subplot(2,1,1)
plt.imshow(label_masks[10])
plt.subplot(2,1,2) 
plt.imshow(final_images[10])

In [ ]:
labels, nlabels = ndimage.label(label_masks[2])

In [ ]:
plt.figure(1,figsize = (10,10))
plt.subplot(2,1,1)
plt.imshow(labels)
plt.subplot(2,1,2)
plt.imshow(label_masks[2])

In [ ]:
#Sanity check
for i in range(len(label_masks)):
    if label_masks[i].shape != final_images[i].shape:
        #print(label_masks[i].shape)
        continue
    else:
        print('no')
        break
    

In [ ]:
#sanity
for i,dirs in enumerate(train_images):
        if (len(np.unique(label_masks[i]))-1) == len(os.listdir(train_dir + dirs + "/" + "masks/")):
            #print(len(np.unique(label_masks[i])))
            #print(len(os.listdir(train_dir + dirs + "/" + "masks/")))
            continue
        else:
            print("no")
            break

In [ ]:
len(label_masks)

In [ ]:
print(final_images[1].shape)
print(label_masks.shape)

In [ ]:
for  i in range(len(label_masks)):
    final_images[i] = resize(final_images[i] , (128,128,4) , preserve_range=True)
    label_masks[i] = resize(label_masks[i] , (128,128,1) , preserve_range=True)
        

In [ ]:
print(final_images.shape)
print(label_masks.shape)   

In [ ]:
final_images = np.stack(final_images)

In [ ]:
label_masks = np.stack(label_masks)

In [ ]:
print(final_images.shape)
print(label_masks.shape)

In [ ]:
plt.figure(1 ,figsize = (5,5))
plt.subplot(2,1,1)
plt.imshow(np.squeeze(label_masks[10]))
plt.subplot(2,1,2) 
plt.imshow(final_images[10])

In [ ]:
for i in range(len(label_masks)):
    label_masks[i][label_masks[i] > 0] = 1
    label_masks[i][label_masks[i]<=0] = 0

In [ ]:
plt.figure(1 ,figsize = (20,20))
plt.subplot(2,1,1)
plt.imshow(np.squeeze(label_masks[20]))
plt.subplot(2,1,2) 
plt.imshow(final_images[20])

In [ ]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = label(y_true_in > 0.5)
    y_pred = label(y_pred_in > 0.5)
    
    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.array(np.mean(metric), dtype=np.float32)

def my_iou_metric(label, pred):
    metric_value = tf.py_func(iou_metric_batch, [label, pred], tf.float32)
    return metric_value

In [ ]:
# Build U-Net model
inputs = Input((128, 128, 4))
s = inputs

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[my_iou_metric])
model.summary()

In [ ]:
earlystopper = EarlyStopping(monitor = 'val_loss',patience=5, verbose=1)

In [ ]:
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', monitor = 'val_loss',verbose=1, save_best_only=True)

In [ ]:
results = model.fit(final_images, label_masks, validation_split=0.1, batch_size=16, epochs=10 ,callbacks=[earlystopper,checkpointer])

In [ ]:
#reading test set
test_dir = os.listdir("../input/stage1_test")

In [ ]:
test_images = []
size_test = []
for dirs in test_dir:
    test_images.append(plt.imread("../input/stage1_test/"+ dirs + "/" + "images"+"/"+os.listdir("../input/stage1_test/"+ dirs + "/" + "images")[0]))
for i in range(len(test_images)):
    size_test.append([test_images[i].shape[0], test_images[i].shape[1]])
    test_images[i] = resize(test_images[i] , (128,128,4))
    

In [ ]:
test_images = np.array(test_images)
test_images = np.stack(test_images)
print(test_images.shape)

In [ ]:
prediction = model.predict(test_images)

In [ ]:
prediction[prediction >= 0.5] = 1
prediction[prediction < 0.5] = 0

In [ ]:
plt.imshow(np.squeeze(prediction[0]))

In [ ]:
plt.imshow(np.squeeze(label_masks[1]))

In [ ]:
preds_test_upsampled = []
for i in range(len(prediction)):
    preds_test_upsampled.append(resize(np.squeeze(prediction[i]), 
                                       (size_test[i][0], size_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
np.unique(prediction[1])

In [ ]:
preds_test_upsampled[1].shape

In [ ]:
preds_test_upsampled[preds_test_upsampled >= 0.5] = 1
preds_test_upsampled[preds_test_upsampled < 0.5] = 0

In [ ]:
plt.imshow(np.squeeze(prediction[1]))

In [ ]:
plt.imshow(np.squeeze(preds_test_upsampled[1]))

In [ ]:
for i in range(len(preds_test_upsampled)):
    preds_test_upsampled[i][preds_test_upsampled[i] > 0.5] = 1
    preds_test_upsampled[i][preds_test_upsampled[i] <= 0.5] = 0

In [ ]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [ ]:
list(rle_encoding(preds_test_upsampled[0]))

In [ ]:
new_test_ids = []
rles = []
for n, id_ in enumerate(test_dir):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

In [ ]:
len(np.unique(new_test_ids))

In [ ]:
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('submit_final.csv', index=False)

In [ ]:
sub